In [1]:
from liblrs_python import Builder, Point, AnchorOnLrm, SegmentOfTraversal
# We start by instantiating a builder that allows us to add each brick of the linear referencing system
builder = Builder()

# The topology
#
# We define a graph made of two nodes ’A’ and ’B’ and a single segment ’s’
# The segment has a geometry
node_a = builder.add_node(id="A", coord=Point(0,0), properties={})
node_b = builder.add_node(id="B", coord=Point(1,0), properties={})
geometry = [Point(0, 0), Point(1, 0)]
segment = builder.add_segment(
    id="s",
    start_node_index=node_a,
    end_node_index=node_b,
    geometry=geometry
)

# The traversal is a list of segment that represents a continuous curve where we want to make the measurements
#
# It can be a the US Route 66, an oil pipeline, etc.
traversal = builder.add_traversal(
    segments=[SegmentOfTraversal(segment_index=segment, reversed=False)],
    traversal_id="traversal"
)

# Anchors are reference points from which all the measurements will be done
# 
# Those are usually milestones, but it could also be a landmark
# The coordonates are not necessarly on the traversal. The library will project them
anchor_0 = builder.add_anchor(id="0", coord=Point(0, 0.1), name="0", properties={})
anchor_1 = builder.add_anchor(id="1", coord=Point(1, 0.1), name="1", properties={})

# The Linear Referencing Method (LRM)
#
# Now that we have
#   - a geometry with the traversal
#   - reference points with the anchors
#
# We need a scale to create the complete LRM
# The distance along the LRM can be in any unit (meters, feet, pixels, steps…)
builder.add_lrm_with_distances(
    id="lrm",
    anchors=[
        AnchorOnLrm(anchor_index=anchor_0, distance_along_lrm=0),
        AnchorOnLrm(anchor_index=anchor_1, distance_along_lrm=800)
    ],
    traversal_index=traversal,
    properties={}
)

# We have everything we need now. Let’s build our LRS file
# 
# We could also save it directly to a file with:
# builder.save(out_file="demo.lrs.bin", properties={})
data = builder.build_data(properties={})


Processing lrm, reversed: false
  On anchor 0, on lrm 0, projected along curve: 0
  On anchor 1, on lrm 800, projected along curve: 1


In [2]:
# Let’s use the the generated data
from liblrs_python import Lrs, LrmScaleMeasure
# First we read the whole file
lrs = Lrs(bytes(data))

# We the search for the LRM with the id "lrm"
lrm = lrs.find_lrm("lrm")

# We can access it’s geometry
print(", ".join([f"({p.x}; {p.y})" for p in lrs.get_lrm_geom(lrm)]))

# Let’s find where is locate the point 200 after the anchor "0"
# The LrmScaleMeasure object allows us to define that position
measure = LrmScaleMeasure(anchor_name="0", scale_offset=200)
position = lrs.resolve(lrm, measure)
print(f"{position.x}; {position.y}")

(0.0; 0.0), (1.0; 0.0)
0.25; 0.0


In [5]:
from liblrs_python import Builder, Point, AnchorOnLrm, SegmentOfTraversal

p = Point(1., 2.)

p.x

1.0